# Fleur de Pain — Agent-Powered Business Assistant

This notebook demonstrates a customer-facing chatbot for Fleur de Pain bakery.

Features:
- Answers questions about the business using RAG from local documents
- Captures leads via function calling
- Logs unknown questions/feedback
- Interactive Gradio chat interface

## 1. Setup and Imports

In [ ]:
import os
import json
from datetime import datetime
from pathlib import Path
import gradio as gr
from openai import OpenAI
from PyPDF2 import PdfReader

# Load API key (fallback to direct file read if dotenv fails)
try:
    from dotenv import load_dotenv
    load_dotenv()
    api_key = os.getenv("OPENAI_API_KEY")
except:
    api_key = None

# Fallback: Read .env file directly
if not api_key or not api_key.startswith('sk-'):
    try:
        with open('.env', 'r') as f:
            for line in f:
                if line.startswith('OPENAI_API_KEY'):
                    api_key = line.split('=', 1)[1].strip()
                    break
    except:
        pass

if not api_key:
    raise ValueError("OPENAI_API_KEY not found. Please create a .env file with your API key.")

# Initialize OpenAI client
client = OpenAI(api_key=api_key)

print("Libraries imported successfully!")
print(f"API key loaded: {api_key[:20]}...{api_key[-10:]}")

## 2. Load Business Context

In [ ]:
def load_business_context():
    """Load business information from PDF and text files."""
    context = ""
    
    # Load PDF
    pdf_path = Path("me/about_business.pdf")
    if pdf_path.exists():
        reader = PdfReader(str(pdf_path))
        pdf_text = ""
        for page in reader.pages:
            pdf_text += page.extract_text() + "\n"
        context += "=== Business Profile (from about_business.pdf) ===\n" + pdf_text + "\n\n"
    
    # Load text summary
    txt_path = Path("me/business_summary.txt")
    if txt_path.exists():
        with open(txt_path, 'r', encoding='utf-8') as f:
            txt_content = f.read()
        context += "=== Business Summary (from business_summary.txt) ===\n" + txt_content + "\n"
    
    return context

# Load the context
BUSINESS_CONTEXT = load_business_context()
print(f"Business context loaded: {len(BUSINESS_CONTEXT)} characters")
print("\nFirst 500 characters:")
print(BUSINESS_CONTEXT[:500])

## 3. Create Logs Directory

In [ ]:
# Ensure logs directory exists
logs_dir = Path("logs")
logs_dir.mkdir(exist_ok=True)
print(f"Logs directory ready: {logs_dir.absolute()}")

## 4. Tool Functions

In [ ]:
def record_customer_interest(email: str, name: str, message: str) -> dict:
    """
    Record customer lead information to JSONL file.
    
    Args:
        email: Customer email or WhatsApp contact
        name: Customer name
        message: Order intent or inquiry details
    
    Returns:
        Confirmation dictionary
    """
    lead_data = {
        "ts": datetime.utcnow().isoformat() + "Z",
        "email": email,
        "name": name,
        "message": message
    }
    
    # Append to JSONL file
    leads_file = Path("logs/leads.jsonl")
    with open(leads_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(lead_data) + "\n")
    
    return {
        "status": "success",
        "message": f"Lead recorded for {name}. Our team will reach out via {email} soon!"
    }


def record_feedback(question: str) -> dict:
    """
    Record unknown questions or feedback to JSONL file.
    
    Args:
        question: The question or feedback from the customer
    
    Returns:
        Confirmation dictionary
    """
    feedback_data = {
        "ts": datetime.utcnow().isoformat() + "Z",
        "question": question
    }
    
    # Append to JSONL file
    feedback_file = Path("logs/feedback.jsonl")
    with open(feedback_file, 'a', encoding='utf-8') as f:
        f.write(json.dumps(feedback_data) + "\n")
    
    return {
        "status": "success",
        "message": "Thank you! We've logged your question for our team to review."
    }

print("Tool functions defined successfully!")

## 5. Define Function Schemas for OpenAI

In [ ]:
# Tool definitions for OpenAI function calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "record_customer_interest",
            "description": "Record customer lead information when they express interest in ordering or want a quote. Call this when you have collected their contact details and order intent.",
            "parameters": {
                "type": "object",
                "properties": {
                    "email": {
                        "type": "string",
                        "description": "Customer's email address or WhatsApp number"
                    },
                    "name": {
                        "type": "string",
                        "description": "Customer's full name"
                    },
                    "message": {
                        "type": "string",
                        "description": "Details about their order intent, items wanted, quantities, dates, etc."
                    }
                },
                "required": ["email", "name", "message"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "record_feedback",
            "description": "Log questions you cannot confidently answer from the business documents, or general customer feedback. Use this when you're uncertain or the question is out of scope.",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The customer's question or feedback"
                    }
                },
                "required": ["question"]
            }
        }
    }
]

print("Function schemas defined!")

## 6. System Prompt

In [ ]:
SYSTEM_PROMPT = f"""You are Fleur de Pain's chat assistant.

GOALS:
1) Answer questions strictly using the business_summary.txt and about_business.pdf.
2) If the user asks for ordering or quotes, collect leads:
   - Ask for name, email (or WhatsApp), desired items, quantities, date/time.
3) If you cannot answer confidently from the docs, call the tool:
   record_feedback(question=the user's question).
4) Stay on brand: warm, concise, transparent about bake times.
5) Emphasize the bakery's policies:
   - Fresh batches every 3 hours; nothing day-old marketed as fresh.
   - Custom cakes require 24-hour notice.
   - Pre-order via WhatsApp; 2-hour delivery windows (when available).

BEHAVIOR:
- Never invent prices or availability if not stated; offer to collect details.
- For cake inquiries, summarize options (flavors/fillings/sizes) and note 24h lead time.
- For bread timing, consult "Bake Times" (if provided) or explain typical windows and that schedule.
- Always encourage sharing contact info (politely) so the team can confirm and schedule.

TOOLS:
- record_customer_interest(email, name, message)
- record_feedback(question)

When you need a lead, call record_customer_interest with all fields you have
and placeholders for missing ones. If a question is out-of-scope or unknown,
call record_feedback.

=== BUSINESS CONTEXT ===
{BUSINESS_CONTEXT}
"""

print("System prompt configured!")

## 7. Chat Handler with Function Calling

In [ ]:
def chat_with_agent(message, history):
    """
    Process user message and return bot response.
    Handles function calling for lead capture and feedback.
    """
    # Build messages from history
    messages = [{"role": "system", "content": SYSTEM_PROMPT}]
    
    # Add conversation history
    for human, assistant in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": assistant})
    
    # Add current message
    messages.append({"role": "user", "content": message})
    
    # Call OpenAI API with function calling
    response = client.chat.completions.create(
        model="gpt-4o",  # Using GPT-4o for best performance
        messages=messages,
        tools=tools,
        tool_choice="auto"
    )
    
    response_message = response.choices[0].message
    
    # Check if the model wants to call a function
    if response_message.tool_calls:
        # Process each tool call
        messages.append(response_message)
        
        for tool_call in response_message.tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)
            
            # Execute the appropriate function
            if function_name == "record_customer_interest":
                function_response = record_customer_interest(
                    email=function_args.get("email"),
                    name=function_args.get("name"),
                    message=function_args.get("message")
                )
            elif function_name == "record_feedback":
                function_response = record_feedback(
                    question=function_args.get("question")
                )
            else:
                function_response = {"error": "Unknown function"}
            
            # Add function response to messages
            messages.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": json.dumps(function_response)
            })
        
        # Get final response after function execution
        second_response = client.chat.completions.create(
            model="gpt-4o",  # Using GPT-4o for best performance
            messages=messages
        )
        
        return second_response.choices[0].message.content
    
    # No function call needed, return direct response
    return response_message.content

print("Chat handler defined!")

## 8. Launch Gradio Interface

In [ ]:
# Create Gradio chat interface
demo = gr.ChatInterface(
    fn=chat_with_agent,
    title="Fleur de Pain — Business Assistant",
    description="Ask me about our fresh-baked goods, menu, ordering, custom cakes, and more!",
    examples=[
        "What types of bread do you offer?",
        "When are fresh batches available?",
        "I need a custom cake for 20 people this Saturday",
        "How do I pre-order for delivery?",
        "Tell me about your viennoiserie",
        "Do you have gluten-free options?"
    ],
    theme=gr.themes.Soft()
)

# Launch the interface
if __name__ == "__main__":
    demo.launch(share=False, server_name="127.0.0.1", server_port=7860)

## 9. Test the Tools Directly (Optional)

In [ ]:
# Test recording a lead
test_lead = record_customer_interest(
    email="test@example.com",
    name="Test Customer",
    message="Interested in 2 sourdough loaves for Saturday morning pickup"
)
print("Lead test:", test_lead)

# Test recording feedback
test_feedback = record_feedback(
    question="Do you offer vegan croissants?"
)
print("Feedback test:", test_feedback)

## 10. View Logged Data (Optional)

In [ ]:
# View leads
leads_file = Path("logs/leads.jsonl")
if leads_file.exists():
    print("=== LEADS ===")
    with open(leads_file, 'r', encoding='utf-8') as f:
        for line in f:
            print(json.loads(line))
else:
    print("No leads logged yet.")

print("\n")

# View feedback
feedback_file = Path("logs/feedback.jsonl")
if feedback_file.exists():
    print("=== FEEDBACK ===")
    with open(feedback_file, 'r', encoding='utf-8') as f:
        for line in f:
            print(json.loads(line))
else:
    print("No feedback logged yet.")